## 全局变量设置

In [1]:
import os
iteration = 1
workspace_dir = "/home/HYX/workspace"
use_ctc_loss = False
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
%cd /home/HYX/workspace/DeepSpeech

[Errno 2] No such file or directory: '/home/HYX/workspace/DeepSpeech'
/home/huangyuxin/workspace/paddle_build/notebook


In [3]:
Feature_dir= os.path.join(workspace_dir, "DeepSpeech/compare/workspace_aishell_asr/feature_dir")

In [4]:
!test -d compare/workspace_aishell_asr || wget -nc https://paddlespeech.bj.bcebos.com/model_alignment/s2t/conformer/check_conformer_layernorm_aishell_paddlespeech.tgz -P ./compare
!test -d compare/workspace_aishell_asr || tar xzvf compare/check_conformer_layernorm_aishell_paddlespeech.tgz -C ./compare
!mkdir -p ./compare/workspace_aishell_asr

--2022-02-28 06:57:31--  https://paddlespeech.bj.bcebos.com/model_alignment/s2t/conformer/check_conformer_layernorm_aishell_paddlespeech.tgz
Connecting to 172.19.57.45:3128... connected.
Proxy request sent, awaiting response... 200 OK
Length: 180879331 (172M) [application/x-gzip]
Saving to: ‘./compare/check_conformer_layernorm_aishell_paddlespeech.tgz’

_conformer_layernor   0%[                    ] 250.84K  51.8KB/s    eta 57m 59s^C
workspace_aishell_asr/
workspace_aishell_asr/conf/
workspace_aishell_asr/conf/conformer.yaml
workspace_aishell_asr/checkpoints/
workspace_aishell_asr/checkpoints/init_whole.pdparams

gzip: stdin: unexpected end of file
tar: Unexpected EOF in archive
tar: Unexpected EOF in archive
tar: Error is not recoverable: exiting now


In [5]:
# 获取用于预测的音频文件
!test -f ./data/demo_01_03.wav || wget -nc https://paddlespeech.bj.bcebos.com/datasets/single_wav/zh/demo_01_03.wav -P ./data/

--2022-02-28 06:57:40--  https://paddlespeech.bj.bcebos.com/datasets/single_wav/zh/demo_01_03.wav
Connecting to 172.19.57.45:3128... connected.
Proxy request sent, awaiting response... ^C


In [6]:
import soundfile
import paddle
import warnings
import numpy as np
import os
warnings.filterwarnings('ignore')

from yacs.config import CfgNode
from paddlespeech.s2t.transform.spectrogram import LogMelSpectrogramKaldi
from paddlespeech.s2t.transform.cmvn import GlobalCMVN
from paddlespeech.s2t.frontend.featurizer.text_featurizer import TextFeaturizer
from paddlespeech.s2t.models.u2 import U2Model
from paddlespeech.s2t.training.optimizer import OptimizerFactory
from paddlespeech.s2t.training.scheduler import LRSchedulerFactory
from matplotlib import pyplot as plt



Error: Can not import avx core while this file exists: /root/miniconda3/lib/python3.7/site-packages/paddle/fluid/core_avx.so
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/root/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-df122f080af1>", line 2, in <module>
    import paddle
  File "/root/miniconda3/lib/python3.7/site-packages/paddle/__init__.py", line 25, in <module>
    from .fluid import monkey_patch_variable
  File "/root/miniconda3/lib/python3.7/site-packages/paddle/fluid/__init__.py", line 36, in <module>
    from . import framework
  File "/root/miniconda3/lib/python3.7/site-packages/paddle/fluid/framework.py", line 37, in <module>
    from . import core
  File "/root/miniconda3/lib/python3.7/site-packages/paddle/fluid/core.py", line 294, in <module>
    raise e
  File "/root/miniconda3/lib/python3.7/site-packages/paddle/fluid/core.py", line 256, in <module>
    from .core_avx import *
ImportError: KeyboardInterrupt: 

During handling of the above exception, another exception

TypeError: object of type 'NoneType' has no len()

In [ ]:
config_path = "compare/workspace_aishell_asr/conf/conformer.yaml" 
decode_config_path = "compare/workspace_aishell_asr/conf/tuning/decode.yaml"
checkpoint_path = os.path.join("compare/workspace_aishell_asr/checkpoints/init_whole.pdparams")
#checkpoint_path = os.path.join(workspace_dir, "wenet/examples/aishell/s0/exp/conformer_noLayernorm/init_no_layernorm2.pdparams")

audio_file = "compare/workspace_aishell_asr/data/demo_01_03.wav"
text_data = "我认为跑步最重要的就是给我带来了身体健康"

# 读取 conf 文件并结构化
conformer_config = CfgNode(new_allowed=True)
conformer_config.merge_from_file(config_path)
decode_config = CfgNode(new_allowed=True)
decode_config.merge_from_file(decode_config_path)
conformer_config.decode = decode_config
conformer_config.decode.decoding_method = decoding_method
conformer_config.vocab_filepath = os.path.join(workspace_dir, "DeepSpeech/compare/workspace_aishell_asr/data/lang_char/vocab.txt")

print(conformer_config)

## 特征处理工具

In [ ]:
# 构建 logmel 特征
logmel_kaldi= LogMelSpectrogramKaldi(
            fs= 16000,
            n_mels= 80,
            n_shift= 160,
            win_length= 400,
            dither= True)

# 特征减均值除以方差
cmvn = GlobalCMVN(
    cmvn_path="compare/workspace_aishell_asr/data/mean_std.json"
)

## 处理特征

In [ ]:
array, _ = soundfile.read(audio_file, dtype="int16")
array = logmel_kaldi(array, train=False)
audio_feature_i = cmvn(array)
audio_len_i = audio_feature_i.shape[0]

text_featurizer = TextFeaturizer(unit_type='char',
                            vocab=conformer_config.vocab_filepath)
text_feat_i = text_featurizer.featurize(text_data)
text_len_i = len(text_feat_i)

audio_len = paddle.to_tensor(audio_len_i)
audio_feature = paddle.to_tensor(audio_feature_i, dtype='float32')
audio_feature = paddle.unsqueeze(audio_feature, axis=0)

text_feat = paddle.to_tensor([text_feat_i])
text_len = paddle.to_tensor([text_feat.shape[1]])

np.save(os.path.join(Feature_dir, "audio_feature_i.npy"), audio_feature_i)
np.save(os.path.join(Feature_dir, "audio_len_i.npy"), audio_len_i)
np.save(os.path.join(Feature_dir, "text_feat_i.npy"), text_feat_i)
np.save(os.path.join(Feature_dir, "text_len_i.npy"), text_len_i)

print ("audio_feature", audio_feature)
print ("audio_len", audio_len)
print ("text_feat", text_feat)
print ("text_len", text_len)

## 构建Conformer模型

In [ ]:
model_conf = conformer_config
# input_dim 存储的是特征的纬度
model_conf.input_dim = 80
# output_dim 存储的字表的长度
model_conf.output_dim = 4233 

# model_conf.model_conf.ctc_weight=0.0
print ("model_conf", model_conf)
model = U2Model.from_config(model_conf)
for name, value in model.named_parameters():
    print (name)

## 加载模型

In [ ]:
model_dict = paddle.load(checkpoint_path)
model.set_state_dict(model_dict)


In [ ]:
train_config = conformer_config
optim_type = train_config.optim
optim_conf = train_config.optim_conf

scheduler_type = train_config.scheduler
scheduler_conf = train_config.scheduler_conf


scheduler_args = {
    "learning_rate": optim_conf.lr,
    "verbose": False,
    "warmup_steps": scheduler_conf.warmup_steps,
    "gamma": scheduler_conf.lr_decay,
    "d_model": model_conf.encoder_conf.output_size,
}
lr_scheduler = LRSchedulerFactory.from_args(scheduler_type,
                                            scheduler_args)

def optimizer_args(
                config,
                parameters,
                lr_scheduler=None, ):
    train_config = config
    optim_type = train_config.optim
    optim_conf = train_config.optim_conf
    scheduler_type = train_config.scheduler
    scheduler_conf = train_config.scheduler_conf
    return {
        "grad_clip": train_config.global_grad_clip,
        "weight_decay": optim_conf.weight_decay,
        "learning_rate": lr_scheduler
        if lr_scheduler else optim_conf.lr,
        "parameters": parameters,
        "epsilon": 1e-9 if optim_type == 'noam' else None,
        "beta1": 0.9 if optim_type == 'noam' else None,
        "beat2": 0.98 if optim_type == 'noam' else None,
    }

optimzer_args = optimizer_args(conformer_config, model.parameters(), lr_scheduler)
optimizer = OptimizerFactory.from_args(optim_type, optimzer_args)

## 进行训练

In [ ]:
for i in range (iteration):
    paddle.set_device('gpu')
    model.eval()
    # encoder_out_model, encoder_mask_model = model._forward_encoder(audio_feature, audio_len)

    # encoder_out_paddle = encoder_out_model.numpy()
    # print ("encoder_out_model", encoder_out_model)
    # np.save("paddle_encoder_out", encoder_out.numpy())

    # print ("encoder_out", encoder_out)
    # print ("encoder_mask", encoder_mask)

    # encoder_out_np = np.load(os.path.join(workspace_dir, "DeepSpeech/compare/wenet/encoder_out.npy")
    # encoder_mask_np = np.load(os.path.join(workspace_dir, "DeepSpeech/compare/wenet/encoder_mask.npy")
    # encoder_out = paddle.to_tensor(encoder_out_np)
    # encoder_mask = paddle.to_tensor(encoder_mask_np)


    # print ("encoder_out_model - encoder_out", encoder_out_model - encoder_out)

    # encoder_out = None
    # encoder_mask = None
    print ("lr", optimizer.get_lr())
    loss, attention_loss, ctc_loss = model(audio_feature, audio_len, text_feat,
                                                        text_len)

    print ("loss", loss)
    print ("attention_loss", attention_loss)
    print ("ctc_loss", ctc_loss)
    if use_ctc_loss == True:
        ctc_loss.backward()
    else:
        attention_loss.backward()
    

    paddle_grad_dict = {}
    count = 0
    for n, p in model.named_parameters():
        count += 1
        msg = f"param grad: {n}: shape: {p.shape} grad: {p.grad}"
        #msg = f"param grad: {n}: shape: {p.shape}"
        print (n)
        if p.grad is not None:
            paddle_grad_dict[n] = p.grad.numpy()
        else:
            pass
        try:
            # 查看一下其中几个受关注的参数的 grad
            if (n == "decoder.decoders.5.norm3.bias"):
                print (msg)
                print (p.grad.numpy())
            
            if (n == "ctc.ctc_lo.bias"): 
                print(msg)
                print (p.grad.numpy())
                np.save("paddle_ctc_bias.npy", p.grad.numpy())
        except:
            pass
    print (count)
    
    optimizer.step()
    optimizer.clear_grad()
    lr_scheduler.step()


## wenet

In [ ]:
%cd /home/HYX/workspace/wenet

In [ ]:
!mkdir -p examples/aishell/s0/exp/conformer_layernorm
!wget https://paddlespeech.bj.bcebos.com/model_alignment/s2t/conformer/wenet_aishell_s0_layernorm_init.tgz
!tar xzvf wenet_aishell_s0_layernorm_init.tgz -C examples/aishell/s0/exp/conformer_layernorm/

In [ ]:
Feature_dir=os.path.join(workspace_dir, "DeepSpeech/compare/workspace_aishell_asr/feature_dir")

In [ ]:
!test -f 20211025_conformer_exp.tar.gz || wget https://wenet-1256283475.cos.ap-shanghai.myqcloud.com/models/wenetspeech/20211025_conformer_exp.tar.gz
!tar xzvf 20211025_conformer_exp.tar.gz 

In [ ]:
from wenet.transformer.asr_model import init_asr_model
from wenet.utils.checkpoint import load_checkpoint, save_checkpoint
from wenet.utils.file_utils import read_symbol_table, read_non_lang_symbols
import yaml
import numpy as np
import os
from torch.utils.data import DataLoader
from wenet.utils.executor import Executor
import torch
import torch.optim as optim
from wenet.utils.scheduler import WarmupLR

In [ ]:
# config_file=os.path.join(workspace_dir, 'wenet/examples/aishell/s0/exp/conformer_noLayernorm/train.yaml')
# checkpoint=os.path.join(workspace_dir, 'wenet/examples/aishell/s0/exp/conformer_noLayernorm/init.pt')
# cmvn_file=os.path.join(workspace_dir, 'wenet/examples/aishell/s0/exp/conformer_noLayernorm/global_cmvn')
# symbol_table=os.path.join(workspace_dir, 'wenet/examples/aishell/s0/exp/conformer_noLayernorm/lang_char.txt')


config_file=os.path.join(workspace_dir, 'wenet/examples/aishell/s0/exp/conformer_layernorm/train.yaml')
checkpoint=os.path.join(workspace_dir, 'wenet/examples/aishell/s0/exp/conformer_layernorm/init.pt')
cmvn_file=os.path.join(workspace_dir, 'wenet/examples/aishell/s0/exp/conformer_layernorm/global_cmvn')
symbol_table=os.path.join(workspace_dir, 'wenet/examples/aishell/s0/exp/conformer_layernorm/lang_char.txt')

symbol_table = read_symbol_table(symbol_table)
vocab_size = len(symbol_table)
with open(config_file, 'r') as fin:
    configs = yaml.load(fin, Loader=yaml.FullLoader)
train_conf = configs['dataset_conf']
input_dim = configs['dataset_conf']['fbank_conf']['num_mel_bins']
configs['input_dim'] = input_dim
configs['output_dim'] = vocab_size
configs['cmvn_file'] = cmvn_file
configs['is_json_cmvn'] = True

print (configs)
model = init_asr_model(configs)
for name, value in model.named_parameters():
    print (name)
infos = load_checkpoint(model, checkpoint)
print (model)
print (infos)
# hack
model.encoder.global_cmvn=None

print (model.parameters())

## 生成特征

In [ ]:
audio_feature_i = np.load(os.path.join(Feature_dir, "audio_feature_i.npy"))
audio_len_i = np.load(os.path.join(Feature_dir, "audio_len_i.npy"))
text_feat_i = np.load(os.path.join(Feature_dir, "text_feat_i.npy"))
text_len_i = np.load(os.path.join(Feature_dir, "text_len_i.npy"))
print (type(audio_len_i))

device = "cuda"
feats = torch.tensor([audio_feature_i], dtype=torch.float32, device = device)
feats_lengths = torch.tensor([int(audio_len_i)], device = device )

target = torch.tensor([text_feat_i], device = device)
target_lengths = torch.tensor([int(text_len_i)], device = device)

print (feats)
print (feats_lengths)
print (target)
print (target_lengths)

In [ ]:
device = 'cuda'
step = -1
model = model.to(device)

optimizer = optim.Adam(model.parameters(), **configs['optim_conf'])
scheduler = WarmupLR(optimizer, **configs['scheduler_conf'])


scheduler.set_step(step)
scheduler.step()

In [ ]:
# for _ in range(10):
#     lr = optimizer.param_groups[0]['lr']
#     print ("lr", lr)
#     optimizer.step()
#     optimizer.zero_grad()
#     scheduler.step()
"""
lr 8e-08
lr 8e-08
lr 1.6e-07
lr 2.4e-07
lr 3.2e-07
lr 4e-07
lr 4.8e-07
lr 5.6e-07
lr 6.4e-07
lr 7.2e-07
"""
    

In [ ]:
for i in range (iteration):    
    model.eval()

#     encoder_out, encoder_mask = model._forward_encoder(feats, feats_lengths)

#     encoder_out_torch = encoder_out.cpu().detach().numpy()


    loss, loss_att, loss_ctc = model(
                                feats, feats_lengths, target, target_lengths)
    print ("loss", loss.cpu().detach().numpy())
    print ("loss_att", loss_att.cpu().detach().numpy())
    print ("loss_ctc", loss_ctc.cpu().detach().numpy())
    if use_ctc_loss == True:
        loss_ctc.backward()
    else:
        loss_att.backward()
    lr = optimizer.param_groups[0]['lr']
    print ("lr", lr)
    
    wenet_grad_dict = {}
    count = 0

    print ("The following are grads:")
    for name,item in model.named_parameters():
        count += 1

        #print (item.grad)
        if item.grad != None:
            print (name)
            wenet_grad_dict[name] = item.grad.cpu().numpy()
        try:
            if (name == "decoder.decoders.5.norm3.bias"):
                print (name, item.grad.cpu().numpy())
            if name == "ctc.ctc_lo.bias":
                print (name, item.grad.cpu().numpy())
        except:
            pass
    print (count)  
    
    # 更新模型
    optimizer.step()
    optimizer.zero_grad()
    scheduler.step()





# 转换wenet（torch）的grad 为paddle格式

In [ ]:
def torch_to_paddle(model_state_dict): 
    paddle_state_dict = {}
    for n, p in model_state_dict.items():
        print(f'-> name: {n} dim: {p.ndim}')
        
        # change norm.mean and norm variance
        name_change=True
        if 'norm.running_mean' in n:
            new_n = n.replace('norm.running_', 'norm._')
        elif 'norm.running_var' in n:
            new_n = n.replace('norm.running_var', 'norm._variance')
        else:
            name_change=False
            new_n = n
        if name_change:
            print(f"\t norm mean/var: {n} -> {new_n}")

        #p = p.cpu().detach().numpy()
        
        # weight which is rank 2, transpose it
        if n.endswith('weight') and p.ndim == 2:
            new_p = p.T
            print(f"\t liner transpose: {n}: {p.shape} -> {new_p.shape}")
        else:
            new_p = p
        
        # text embedding layer
        if 'decoder.embed.0.weight' in n:
            new_p = p

        if 'global_cmvn.mean' in n:
            print("\t cmvn:", p, p.dtype)
        if 'global_cmvn.istd' in n:
            print("\t istd:", p, p.dtype)

        paddle_state_dict[new_n] = new_p
    return paddle_state_dict
wenet_to_paddle_grad_dict = torch_to_paddle(wenet_grad_dict)


In [ ]:
for key in paddle_grad_dict:
    torch_grad = wenet_to_paddle_grad_dict[key]
    paddle_grad = paddle_grad_dict[key]
    
    res =np.allclose(torch_grad, paddle_grad, rtol=1e-5, atol=1e-6)
    if (res == False):
        print (key)
#         print ("torch_grad", torch_grad)
#         print ("paddle_grad", paddle_grad)
        print (res)
        print (key, "max_grad_diff", np.max(abs(torch_grad - paddle_grad)))
        print ("mean grad diff", np.mean(abs(torch_grad - paddle_grad)))
        print ("mean", np.mean(abs(torch_grad)))
        print ("has false")
        
#     if (key == "encoder.encoders.0.conv_module.pointwise_conv1.weight"):
#         print ("torch_grad")
#         print (torch_grad)
#         print ("paddle_grad")
#         print (paddle_grad)
#     if (key == "ctc.ctc_lo.bias"):
#         print (torch_grad)
#         print (paddle_grad)